# TODO

- [x] IDENTIFICAÇÃO DAS CIDADES LITORÂNEAS  
- [x] LEVANTAMENTO DA QUANTIDADE DE HOSPITAIS  
- [ ] LEVANTAMENTO DA SEGURANÇA  
- [ ] LEVANTAMENTO DA MÉDIA DE ALUGUÉIS  
- [x] LEVANTAMENTO DOS PREÇOS DE ALIMENTO

# IMPORTAÇÃO DE BIBLIOTECAS

In [1]:
import polars as pl
import os
import duckdb

In [2]:
AUXILIAR_PATH = r'finding-home/finding_home/auxiliar/'

# IMPORTAÇÃO DE DADOS

## CIDADES

In [3]:
from unicodedata import normalize 
def custnorm(In_series):
    for i, x in enumerate(In_series):
        newvalue = normalize('NFKD',x).encode('ascii', errors='ignore').decode('utf-8')
        if newvalue != x:
            In_series[i]=newvalue
    return In_series

In [4]:
cities = pl.read_csv(os.path.join(AUXILIAR_PATH, 'CIDADES_LITORANEAS_IBGE.csv'), separator=';') \
           .filter(pl.col('NM_REGIAO').is_in(['Sul', 'Sudeste', 'Nordeste'])) \
           .with_columns(pl.col('NM_MUN').str.to_uppercase().alias('MUNICIPIO')) \
           .with_columns(pl.col('MUNICIPIO').map(custnorm).alias('MUNICIPIO'))

/tmp/ipykernel_890/3017128636.py:4: DeprecationWarning: `map` is deprecated. It has been renamed to `map_batches`.
  .with_columns(pl.col('MUNICIPIO').map(custnorm).alias('MUNICIPIO'))


## HOSPITAIS

In [5]:
hospitals = pl.read_csv(os.path.join(AUXILIAR_PATH, 'hospitais.csv'), separator='|', encoding='cp1252', truncate_ragged_lines=True)

In [6]:
hospitals_stat = duckdb.sql("""
                            SELECT 
                                    REGIAO
                                ,   UF
                                ,   MUNICIPIO
                                ,   DESC_NATUREZA_JURIDICA
                                ,   SUM(CASE WHEN DS_TIPO_UNIDADE == 'HOSPITAL GERAL' THEN 1 ELSE 0 END) AS HOSPITAL_GERAL
                                ,   SUM(CASE WHEN DS_TIPO_UNIDADE == 'PRONTO SOCORRO GERAL' THEN 1 ELSE 0 END) AS PRONTO_SOCORRO_GERAL 
                                ,   SUM(CASE WHEN DS_TIPO_UNIDADE == 'UNIDADE MISTA' THEN 1 ELSE 0 END) AS UNIDADE_MISTA
                            
                            FROM hospitals
                            GROUP BY REGIAO
                                ,   UF
                                ,   MUNICIPIO
                                ,   DESC_NATUREZA_JURIDICA

                            ORDER BY REGIAO
                                ,   UF
                                ,   MUNICIPIO
                                ,   DESC_NATUREZA_JURIDICA
                            
                            """).to_df()

In [7]:
hospitals_stat.head()

,REGIAO,UF,MUNICIPIO,DESC_NATUREZA_JURIDICA,HOSPITAL_GERAL,PRONTO_SOCORRO_GERAL,UNIDADE_MISTA
0,CENTRO-OESTE,DF,BRASILIA,HOSPITAL_FILANTRÓPICO,4.0,0.0,0.0
1,CENTRO-OESTE,DF,BRASILIA,HOSPITAL_PRIVADO,26.0,0.0,0.0
2,CENTRO-OESTE,DF,BRASILIA,HOSPITAL_PÚBLICO,16.0,0.0,0.0
3,CENTRO-OESTE,GO,ABADIANIA,HOSPITAL_PÚBLICO,0.0,0.0,1.0
4,CENTRO-OESTE,GO,ACREUNA,HOSPITAL_PRIVADO,2.0,0.0,0.0


# ALIMENTOS

In [8]:
food_filter = [
                'ARROZ LONGO FINO BENEFICIADO TIPO 1 (5 kg)'
                , 'FEIJÃO COMUM CORES TIPO 1 (kg)'
                , 'FEIJÃO COMUM PRETO TIPO 1 (kg)'
                , 'CARNE BOVINA ACÉM (kg)'
                , 'CARNE BOVINA COSTELA (kg)'
                , 'CARNE BOVINA PONTA DE AGULHA (kg)'
                , 'CARNE DE FRANGO CONGELADO (kg)'
                , 'CARNE DE FRANGO RESFRIADO (kg)'
                , 'CARNE SUÍNA CONGELADO PERNIL COM OSSO (kg)'
                , 'CARNE SUÍNA RESFRIADO PERNIL COM OSSO (kg)'
                , 'OLEO DE SOJA REFINADO (900 ml)'
                , 'LEITE DE VACA LONGA VIDA INTEGRAL (l)'
                , 'MACARRÃO ESPAGUETE SÊMOLA COM OVOS (500 g)'
                , 'MACARRÃO ESPAGUETE SÊMOLA COM OVOS (kg)'
                , 'SAL (kg)'
                ]

In [9]:
food = pl.read_csv(os.path.join(AUXILIAR_PATH, 'ALIMENTOS_CONAB.csv'), separator=';') \
         .melt(id_vars=['UF', 'PRODUTO'], value_vars=pl.selectors.numeric(), variable_name='MES_REF', value_name='PRECO') \
         .filter(pl.col('PRODUTO').is_in(food_filter)) \
         .pivot(values='PRECO', columns='PRODUTO', index='UF', aggregate_function='mean') \
         .fill_null(0.0)
         

In [10]:
food.head()

UF,ARROZ LONGO FINO BENEFICIADO TIPO 1 (5 kg),CARNE BOVINA ACÉM (kg),CARNE BOVINA COSTELA (kg),CARNE BOVINA PONTA DE AGULHA (kg),CARNE DE FRANGO CONGELADO (kg),CARNE DE FRANGO RESFRIADO (kg),CARNE SUÍNA CONGELADO PERNIL COM OSSO (kg),CARNE SUÍNA RESFRIADO PERNIL COM OSSO (kg),FEIJÃO COMUM CORES TIPO 1 (kg),FEIJÃO COMUM PRETO TIPO 1 (kg),LEITE DE VACA LONGA VIDA INTEGRAL (l),MACARRÃO ESPAGUETE SÊMOLA COM OVOS (500 g),MACARRÃO ESPAGUETE SÊMOLA COM OVOS (kg),OLEO DE SOJA REFINADO (900 ml),SAL (kg)
str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""ES""",20.27,0.0,22.499091,0.0,10.084545,13.049091,16.122727,0.0,8.373636,6.773636,4.796364,4.995455,0.0,6.657273,2.388182
"""GO""",23.567273,0.0,0.0,17.223636,7.423636,7.917273,0.0,13.353636,8.934545,9.976364,5.564545,0.0,9.150909,6.314545,3.187273
"""MA""",23.606364,0.0,0.0,0.0,0.0,0.0,0.0,14.990909,7.834545,7.345455,5.643636,4.421818,0.0,7.06,1.378182
"""MG""",23.878182,0.0,0.0,22.383636,12.462727,12.236364,0.0,19.38,8.533636,8.107273,4.870909,4.923636,0.0,6.452727,0.0
"""MS""",24.448182,0.0,0.0,16.530909,9.504545,11.877273,0.0,12.293636,9.032727,8.541818,5.535455,0.0,10.160909,6.482727,0.0


# SEGURANÇA

## TRANSPORTE

In [11]:
ob_transporte = pl.read_csv(os.path.join(AUXILIAR_PATH, 'bitos-em-acidentes-de-transporte.csv'), separator=';') \
                  .filter(pl.col('período') > 2018) \
                  .unique() \
                  .group_by(['cod', 'nome']).agg( \
                                pl.col('valor').min().alias('MIN_OBITOS_CARRO') \
                              , pl.col('valor').mean().alias('MEDIA_OBITOS_CARRO') \
                              , pl.col('valor').max().alias('MAX_OBITOS_CARRO') \
                              , pl.col('período').n_unique().alias('QTD_HISTORICO_CARRO') \
                              , pl.when(pl.col('valor') == pl.col('valor').max()) \
                                                .then(pl.col('período')) \
                                                .otherwise(pl.lit(0)).alias('ANO_MAIOR_CARRO')) \
                            .with_columns(pl.col('ANO_MAIOR_CARRO').cast(pl.Array(inner=pl.Int64, width=3)).arr.max().alias('ANO_MAIOR_CARRO'))

In [12]:
ob_transporte.head()

cod,nome,MIN_OBITOS_CARRO,MEDIA_OBITOS_CARRO,MAX_OBITOS_CARRO,QTD_HISTORICO_CARRO,ANO_MAIOR_CARRO
i64,str,i64,f64,i64,u32,i64
4307054,"""Ernestina""",0,1.0,3,3,2019
4305116,"""Centenário""",0,0.333333,1,3,2020
4300307,"""Alecrim""",1,1.333333,2,3,2019
3519303,"""Ibaté""",5,7.0,8,3,2021
3515608,"""Fernando Prest…",0,0.666667,2,3,2020


## MORTES

In [13]:
homicidios = pl.read_csv(os.path.join(AUXILIAR_PATH, 'homicidios.csv'), separator=';') \
                  .filter(pl.col('período') > 2018) \
                  .unique() \
                  .group_by(['cod', 'nome']).agg( \
                                pl.col('valor').min().alias('MIN_OBITOS_HOMICIDIOS') \
                              , pl.col('valor').mean().alias('MEDIA_OBITOS_HOMICIDIOS') \
                              , pl.col('valor').max().alias('MAX_OBITOS_HOMICIDIOS') \
                              , pl.col('período').n_unique().alias('QTD_HISTORICO_HOMICIDIOS') \
                              , pl.when(pl.col('valor') == pl.col('valor').max()) \
                                                .then(pl.col('período')) \
                                                .otherwise(pl.lit(0)).alias('ANO_MAIOR_HOMICIDIOS')) \
                             .with_columns(pl.col('ANO_MAIOR_HOMICIDIOS').cast(pl.Array(inner=pl.Int64, width=3)).arr.max().alias('ANO_MAIOR_HOMICIDIOS'))



In [14]:
homicidios.head()

cod,nome,MIN_OBITOS_HOMICIDIOS,MEDIA_OBITOS_HOMICIDIOS,MAX_OBITOS_HOMICIDIOS,QTD_HISTORICO_HOMICIDIOS,ANO_MAIOR_HOMICIDIOS
i64,str,i64,f64,i64,u32,i64
1100924,"""Chupinguaia""",2,3.333333,6,3,2021
1200708,"""Xapuri""",2,6.333333,14,3,2020
2100055,"""Açailândia""",45,55.0,66,3,2020
2100477,"""Alto Alegre do…",3,4.666667,6,3,2019
2107001,"""Montes Altos""",1,2.666667,4,3,2021


## MORTES ARMA

In [33]:
hom_arma = pl.read_csv(os.path.join(AUXILIAR_PATH, 'homicidios-por-armas-de-fogo.csv'), separator=';') \
                  .filter(pl.col('período') > 2018) \
                  .unique() \
                  .group_by(['cod', 'nome']).agg( \
                                pl.col('valor').min().alias('MIN_OBITOS_HOM_ARMA') \
                              , pl.col('valor').mean().alias('MEDIA_OBITOS_HOM_ARMA') \
                              , pl.col('valor').max().alias('MAX_OBITOS_HOM_ARMA') \
                              , pl.col('período').n_unique().alias('QTD_HISTORICO_HOM_ARMA') \
                              , pl.when(pl.col('valor') == pl.col('valor').max()) \
                                                .then(pl.col('período')) \
                                                .otherwise(pl.lit(0)).alias('ANO_MAIOR_HOM_ARMA')) \
                            .filter(pl.col('QTD_HISTORICO_HOM_ARMA') > 2) \
                            .with_columns(pl.col('ANO_MAIOR_HOM_ARMA').cast(pl.Array(inner=pl.Int64, width=3)).arr.max().alias('ANO_MAIOR_HOM_ARMA'))

In [36]:
hom_arma.head()

cod,nome,MIN_OBITOS_HOM_ARMA,MEDIA_OBITOS_HOM_ARMA,MAX_OBITOS_HOM_ARMA,QTD_HISTORICO_HOM_ARMA,ANO_MAIOR_HOM_ARMA
i64,str,i64,f64,i64,u32,i64
5218789,"""Rio Quente""",0,1.333333,3,3,2019
2607950,"""Jaqueira""",2,4.333333,8,3,2019
2601300,"""Barra de Guabi…",4,5.666667,8,3,2020
2407252,"""Major Sales""",0,0.333333,1,3,2019
2206407,"""Monsenhor Gil""",0,0.333333,1,3,2019


# UNIÃO CIDADES & HOSPITAIS

In [40]:
city_hosp = duckdb.sql("""
           SELECT 
              CT.CD_MUN
            , CT.NM_MUN
            , CT.AREA_KM2
            , CT.CD_UF
            , CT.NM_UF
            , HS.*
            , FD.*
            , HARMA.MIN_OBITOS_HOM_ARMA
            , HARMA.MEDIA_OBITOS_HOM_ARMA
            , HARMA.MAX_OBITOS_HOM_ARMA
            , HARMA.QTD_HISTORICO_HOM_ARMA
            , HARMA.ANO_MAIOR_HOM_ARMA
                       
            , HOM.MIN_OBITOS_HOMICIDIOS
            , HOM.MEDIA_OBITOS_HOMICIDIOS
            , HOM.MAX_OBITOS_HOMICIDIOS
            , HOM.QTD_HISTORICO_HOMICIDIOS
            , HOM.ANO_MAIOR_HOMICIDIOS

            , OBTRANS.MIN_OBITOS_CARRO
            , OBTRANS.MEDIA_OBITOS_CARRO
            , OBTRANS.MAX_OBITOS_CARRO
            , OBTRANS.QTD_HISTORICO_CARRO
            , OBTRANS.ANO_MAIOR_CARRO

           FROM hospitals_stat AS HS

           JOIN cities AS CT
           ON CT.SIGLA_UF = HS.UF
           AND CT.MUNICIPIO = HS.MUNICIPIO
           
           JOIN food FD
           ON FD.UF = HS.UF
                       
           JOIN hom_arma HARMA
           ON HARMA.cod = CT.CD_MUN
                       
           JOIN homicidios HOM
           ON HOM.cod = CT.CD_MUN

           JOIN ob_transporte OBTRANS
           ON OBTRANS.cod = CT.CD_MUN
    
                      
           """).to_df()

In [41]:
city_hosp.head()

,CD_MUN,NM_MUN,AREA_KM2,CD_UF,NM_UF,REGIAO,UF,MUNICIPIO,DESC_NATUREZA_JURIDICA,HOSPITAL_GERAL,...,MIN_OBITOS_HOMICIDIOS,MEDIA_OBITOS_HOMICIDIOS,MAX_OBITOS_HOMICIDIOS,QTD_HISTORICO_HOMICIDIOS,ANO_MAIOR_HOMICIDIOS,MIN_OBITOS_CARRO,MEDIA_OBITOS_CARRO,MAX_OBITOS_CARRO,QTD_HISTORICO_CARRO,ANO_MAIOR_CARRO
0,2702306,Coruripe,897.800,27,Alagoas,NORDESTE,AL,CORURIPE,HOSPITAL_PRIVADO,1.0,...,11,16.333333,20,3,2019,9,14.333333,19,3,2020
1,2704302,Maceió,509.320,27,Alagoas,NORDESTE,AL,MACEIO,HOSPITAL_FILANTRÓPICO,4.0,...,329,363.666667,400,3,2020,105,111.666667,124,3,2020
2,2704302,Maceió,509.320,27,Alagoas,NORDESTE,AL,MACEIO,HOSPITAL_PRIVADO,12.0,...,329,363.666667,400,3,2020,105,111.666667,124,3,2020
3,2704302,Maceió,509.320,27,Alagoas,NORDESTE,AL,MACEIO,HOSPITAL_PÚBLICO,4.0,...,329,363.666667,400,3,2020,105,111.666667,124,3,2020
4,2900801,Alcobaça,1477.929,29,Bahia,NORDESTE,BA,ALCOBACA,HOSPITAL_PÚBLICO,1.0,...,7,15.666667,29,3,2021,1,5.666667,9,3,2021


In [38]:
duckdb.sql("SELECT * FROM city_hosp WHERE CD_MUN = 2702306")

┌─────────┬──────────┬──────────┬───────┬───┬──────────────────────┬──────────────────────┬────────────────────┐
│ CD_MUN  │  NM_MUN  │ AREA_KM2 │ CD_UF │ … │ MACARRÃO ESPAGUETE…  │ OLEO DE SOJA REFIN…  │      SAL (kg)      │
│  int64  │ varchar  │  double  │ int64 │   │        double        │        double        │       double       │
├─────────┼──────────┼──────────┼───────┼───┼──────────────────────┼──────────────────────┼────────────────────┤
│ 2702306 │ Coruripe │    897.8 │    27 │ … │                  0.0 │                 7.61 │ 1.8645454545454543 │
├─────────┴──────────┴──────────┴───────┴───┴──────────────────────┴──────────────────────┴────────────────────┤
│ 1 rows                                                                                  28 columns (7 shown) │
└──────────────────────────────────────────────────────────────────────────────────────────────────────────────┘

In [39]:
duckdb.sql("SELECT * FROM hom_arma WHERE cod = 2702306")

┌─────────┬──────────┬─────────────────────┬───┬─────────────────────┬──────────────────────┬────────────────────┐
│   cod   │   nome   │ MIN_OBITOS_HOM_ARMA │ … │ MAX_OBITOS_HOM_ARMA │ QTD_HISTORICO_HOM_…  │ ANO_MAIOR_HOM_ARMA │
│  int64  │ varchar  │        int64        │   │        int64        │        uint32        │       int64        │
├─────────┼──────────┼─────────────────────┼───┼─────────────────────┼──────────────────────┼────────────────────┤
│ 2702306 │ Coruripe │                   4 │ … │                  20 │                    3 │               2019 │
├─────────┴──────────┴─────────────────────┴───┴─────────────────────┴──────────────────────┴────────────────────┤
│ 1 rows                                                                                     7 columns (6 shown) │
└────────────────────────────────────────────────────────────────────────────────────────────────────────────────┘

# LINKS AUXILIARES

- [API IBGE CIDADES](https://servicodados.ibge.gov.br/api/v3/agregados/6579/periodos/2021/variaveis/9324?localidades=N6[1100015])
- [API IBGE AGREGADA](https://servicodados.ibge.gov.br/api/docs/agregados?versao=3)
- [ALIMENTOS CONAB](https://sisdep.conab.gov.br/precosiagroweb/)
- [ATLAS DA VIOLENCIA](https://www.ipea.gov.br/atlasviolencia/filtros-series)
